# OpenAPI Integration

> How FastStripe leverages Stripe's OpenAPI specification

This document explains how FastStripe uses Stripe's OpenAPI specification to provide automatic API coverage and self-documenting methods.

## What is OpenAPI?

OpenAPI (formerly Swagger) is a specification for describing REST APIs. It defines:

- **Endpoints**: Available URLs and HTTP methods
- **Parameters**: Required and optional parameters for each endpoint
- **Schemas**: Data structures for requests and responses
- **Documentation**: Descriptions and examples

Stripe maintains an official OpenAPI specification that describes their entire API.

## Why Use OpenAPI?

Traditional SDK approaches require manual maintenance:

```python
# Manual approach - requires constant updates
class StripeClient:
    def create_customer(self, email, name=None, phone=None, ...):
        # Hard-coded parameters
        # Manual documentation
        # Prone to becoming outdated
```

OpenAPI-driven approach provides:

1. **Automatic updates**: Always reflects latest API
2. **Complete coverage**: No missing endpoints
3. **Accurate documentation**: Pulled directly from Stripe
4. **Parameter validation**: Based on official schemas

## Stripe's OpenAPI Structure

Let's examine how Stripe organizes their API specification:

In [ ]:
import httpx

# Fetch Stripe's OpenAPI spec
stripe_openapi_url = 'https://raw.githubusercontent.com/stripe/openapi/refs/heads/master/openapi/spec3.json'
stripe_spec = httpx.get(stripe_openapi_url).json()

# Top-level structure
print("OpenAPI spec keys:", list(stripe_spec.keys()))
print(f"API version: {stripe_spec['info']['version']}")
print(f"Number of paths: {len(stripe_spec['paths'])}")

### Examining an Endpoint

Let's look at how a typical endpoint is defined:

In [ ]:
# Example: Customer creation endpoint
customers_path = stripe_spec['paths']['/v1/customers']
create_customer = customers_path['post']

print(f"Operation ID: {create_customer['operationId']}")
print(f"Summary: {create_customer['summary']}")
print(f"Description: {create_customer['description'][:100]}...")

# Parameters from request body schema
schema = create_customer['requestBody']['content']['application/x-www-form-urlencoded']['schema']
properties = schema['properties']
print(f"\nAvailable parameters: {len(properties)}")
print("First 5 parameters:", list(properties.keys())[:5])

### Parameter Information

Each parameter includes rich metadata:

In [ ]:
# Example: email parameter
email_param = properties['email']
print("Email parameter:")
print(f"  Type: {email_param.get('type')}")
print(f"  Max length: {email_param.get('maxLength')}")
print(f"  Description: {email_param.get('description')}")

# Example: address parameter (nested object)
address_param = properties['address']
print("\nAddress parameter:")
print(f"  Type: {address_param['anyOf'][0]['type']}")
print(f"  Description: {address_param.get('description')}")
if 'properties' in address_param['anyOf'][0]:
    address_props = address_param['anyOf'][0]['properties']
    print(f"  Nested fields: {list(address_props.keys())}")

## FastStripe's Processing Pipeline

Here's how FastStripe transforms the OpenAPI spec into Python methods:

### 1. Endpoint Extraction

In [ ]:
def stripe_endpoints(spec: dict):
    """Extract all endpoints from OpenAPI spec"""
    endpoints = []
    
    for path, methods in spec['paths'].items():
        for verb, details in methods.items():
            # Extract basic info
            op_id = details.get('operationId', '')
            summary = details.get('summary', '')
            
            # Extract query parameters
            query_params = [
                dict(name=p['name'], description=p.get('description', ''))
                for p in details.get('parameters', []) 
                if p.get('in') == 'query'
            ]
            
            # Extract body parameters
            body_params = []
            if 'requestBody' in details:
                schema_path = ['requestBody', 'content', 'application/x-www-form-urlencoded', 'schema', 'properties']
                schema = nested_idx(details, *schema_path) or {}
                body_params = [
                    dict(name=k, description=v.get('description', ''))
                    for k, v in schema.items()
                ]
            
            all_params = query_params + body_params
            endpoints.append(dict(
                path=path, 
                verb=verb, 
                op_id=op_id, 
                summary=summary, 
                params=all_params
            ))
    
    return endpoints

### 2. Method Generation

Each endpoint becomes a Python function with proper signature and documentation:

In [ ]:
from inspect import Parameter, Signature

def _mk_func(path, verb, param_info, summary, hdrs={}):
    """Generate Python function from endpoint definition"""
    
    # Create function signature
    sig_params = [
        Parameter(param['name'], Parameter.KEYWORD_ONLY, default=None) 
        for param in param_info
    ]
    
    # Generate docstring
    param_docs = '\n'.join(
        f"    {param['name']}: {param['description']}" 
        for param in param_info
    )
    docstring = f"{summary}\n\nParameters:\n{param_docs}" if param_docs else summary
    
    # Create the actual function
    def method(**kwargs):
        response = getattr(httpx, verb)(
            stripe_api_url + path, 
            headers=hdrs,
            params=_flatten_data(kwargs)
        )
        return dict2obj(response.json())
    
    # Attach metadata
    method.__signature__ = Signature(sig_params)
    method.__doc__ = docstring
    
    return method

### 3. Resource Grouping

Methods are organized by resource type based on operation IDs:

In [ ]:
import re
from collections import defaultdict

def _parse_operation_id(op_id):
    """Parse PostCustomers -> ('customers', 'create')"""
    parts = re.findall(r'[A-Z][a-z]*', op_id)
    verb, *resource_parts = [p.lower() for p in parts]
    
    resource = '_'.join(resource_parts) if resource_parts else 'misc'
    method_name = {
        'post': 'create',
        'get': 'fetch',
        'delete': 'delete'
    }.get(verb, verb)
    
    return resource, method_name

# Group endpoints by resource
eps = stripe_endpoints(stripe_spec)
groups = defaultdict(list)

for ep in eps:
    resource, method_name = _parse_operation_id(ep['op_id'])
    groups[resource].append((ep['path'], ep['verb'], method_name, ep['summary'], ep['params']))

print(f"Found {len(groups)} resource groups")
print("Resource groups:", list(groups.keys())[:10])

## Benefits of OpenAPI Integration

### 1. Complete API Coverage

FastStripe automatically supports all Stripe endpoints without manual implementation:

In [ ]:
# All these work automatically:
sapi.customers.create()          # POST /v1/customers
sapi.products.create()           # POST /v1/products
sapi.prices.create()             # POST /v1/prices
sapi.checkout_sessions.create()  # POST /v1/checkout/sessions
sapi.payment_intents.create()    # POST /v1/payment_intents
sapi.subscriptions.create()      # POST /v1/subscriptions
sapi.invoices.create()           # POST /v1/invoices
sapi.coupons.create()            # POST /v1/coupons
# ... and 100+ more endpoints

### 2. Self-Documenting Methods

Every method includes parameter descriptions from Stripe's official docs:

In [ ]:
# IDE will show all parameters with descriptions
sapi.customers.create(
    email='user@example.com',        # Customer's email address
    name='John Doe',                 # Customer's full name
    phone='+1234567890',            # Customer's phone number
    address={
        'line1': '123 Main St',       # Address line 1
        'city': 'San Francisco',      # City
        'state': 'CA',               # State
        'postal_code': '94105',      # Postal code
        'country': 'US'              # Country
    },
    metadata={                        # Key-value pairs for custom data
        'user_id': '12345'
    }
)

### 3. Automatic Updates

When Stripe adds new endpoints or parameters, FastStripe automatically supports them:

```python
# If Stripe adds a new 'priority' parameter to customers
# FastStripe will immediately support it:
customer = sapi.customers.create(
    email='user@example.com',
    priority='high'  # New parameter automatically available
)
```

## Handling OpenAPI Complexity

### Nested Parameters

OpenAPI can define complex nested structures. FastStripe handles these elegantly:

In [ ]:
# OpenAPI defines address as:
# {
#   "address": {
#     "anyOf": [
#       {
#         "type": "object",
#         "properties": {
#           "line1": {"type": "string"},
#           "city": {"type": "string"}
#         }
#       }
#     ]
#   }
# }

# FastStripe allows natural Python syntax:
sapi.customers.create(
    email='user@example.com',
    address={
        'line1': '123 Main St',
        'city': 'San Francisco'
    }
)

### Parameter Validation

While FastStripe doesn't implement client-side validation, the OpenAPI spec provides the foundation for future enhancements:

- **Type information**: String, integer, boolean, array, object
- **Constraints**: Max length, min/max values, enum options
- **Required fields**: Which parameters are mandatory
- **Format validation**: Email, URL, date formats

## Limitations and Considerations

### Runtime Dependency

FastStripe requires internet access to fetch the OpenAPI spec. Future versions could:

- Cache the spec locally
- Bundle a fallback spec
- Provide offline mode

### Error Handling

OpenAPI doesn't provide detailed error information. FastStripe relies on Stripe's HTTP error responses.

### Complex Workflows

OpenAPI describes individual endpoints, not multi-step workflows. FastStripe's high-level methods (`one_time_payment`, `subscription`) bridge this gap.

## Future Enhancements

### Type Hints

Generate dynamic type hints from OpenAPI schemas:

```python
def create_customer(
    *,
    email: Optional[str] = None,
    name: Optional[str] = None,
    address: Optional[CustomerAddress] = None
) -> Customer:
    ...
```

### Client-Side Validation

Validate parameters before sending requests:

```python
# Catch errors early
sapi.customers.create(email='invalid-email')  # ValidationError
```

### Response Models

Generate typed response models from OpenAPI schemas for better IDE support and runtime safety.